In [15]:
import numpy as np
from ptsa.data.filters.MorletWaveletFilter import MorletWaveletFilter
from ptsa.data.readers import EEGReader
from ptsa.data.filters import MonopolarToBipolarMapper
from ptsa.data.filters import ResampleFilter
from ptsa.data.filters import ButterworthFilter
# from ptsa.data.readers.TalReader import TalReader
from ptsa.data.readers import TalReader
# from ptsa.data.readers.IndexReader import JsonIndexReader
from ptsa.data.readers import JsonIndexReader
from ptsa.data.readers import BaseEventReader
from ptsa.data.TimeSeriesX import TimeSeriesX as TimeSeries
import h5py
import os


In [9]:
run_on_rhino = True

if run_on_rhino:
    rhino_mount = ''
else:
    rhino_mount = '/home/ctw/fusemounts/rhino'

# evfile_date = '20180109'
output_root = '/scratch/cweidema/bootcamp/tmpdat'

pow_params = {
    'catFR1': {
        'encoding': {
            'output_path': (rhino_mount+output_root+'/RAM/RAM_catFR/'+
                            'RAM_catFR1_power/encoding/hdf5_files_sess'),
            # 'ev_file': (rhino_mount+
            #             '/home1/cweidema/Christoph/Analyses/RAM/RAM_catFR/'+
            #             'data/RAM_catFR1_events'+evfile_date+'_jim.npy'),
            'mirror': False,
            'start_time': -0.5,
            'end_time': 2.1,
            'buf': 1.0,
            # 'evfilt': ev['type']=='WORD'
            'ev_type': ['WORD']
        },
        'retrieval': {
            'output_path': (rhino_mount+output_root+'/scratch/cweidema/RAM/RAM_catFR/'+
                            'RAM_catFR1_power/retrieval/hdf5_files_sess'),
            # 'ev_file': (rhino_mount+
            #             '/home1/cweidema/Christoph/Analyses/RAM/RAM_catFR/'+
            #             'data/RAM_catFR1_events'+evfile_date+'_jim.npy'),
            'mirror': True,
            'start_time': -0.9,
            'end_time': -0.1,
            'buf': 0.76,
            # 'evfilt': ((ev['type']=='REC_WORD') |
            #            (ev['type']=='REC_BASE'))
            'ev_type': ['REC_WORD', 'REC_BASE']
        }
    },
    'FR1': {
        'encoding': {
            'output_path': (rhino_mount+output_root+'/scratch/cweidema/RAM/RAM_FR/'+
                            'RAM_FR1_power/encoding/hdf5_files_sess'),
            # 'ev_file': (rhino_mount+
            #             '/home1/cweidema/Christoph/Analyses/RAM/RAM_FR/'+
            #             'data/RAM_FR1_events'+evfile_date+'_jim.npy'),
            'mirror': False,
            'start_time': -0.5,
            'end_time': 2.1,
            'buf': 1.0,
            # 'evfilt': ev['type']=='WORD'
            'ev_type': ['WORD']
        },
        'retrieval': {
            'output_path': (rhino_mount+output_root+'/scratch/cweidema/RAM/RAM_FR/'+
                            'RAM_FR1_power/retrieval/hdf5_files_sess'),
            # 'ev_file': (rhino_mount+
            #             '/home1/cweidema/Christoph/Analyses/RAM/RAM_FR/'+
            #             'data/RAM_FR1_events'+evfile_date+'_jim.npy'),
            'mirror': True,
            'start_time': -0.9,
            'end_time': -0.1,
            'buf': 0.76,
            # 'evfilt': ((ev['type']=='REC_WORD') |
            #            (ev['type']=='REC_BASE'))
            'ev_type': ['REC_WORD', 'REC_BASE']
        }
    },
}



In [4]:
LINEFILT = [58, 62]
POWHZ = 50.
FREQS = np.logspace(np.log10(3),np.log10(180), 8)

jr = JsonIndexReader(rhino_mount + '/protocols/r1.json')

atlas_defaults = {'id': ('', np.str), 'region': ('', np.str), 'x': (np.nan, np.float),
                  'y': (np.nan, np.float), 'z': (np.nan, np.float)}


# not sure what's wrong with this subject
bad_subjects = ['R1135E', 'R1221P', 'R1247P', 'R1269E', 'R1348J', 'R1366J', 'R1370E', 'R1372C',
                'R1004D', 'R1226D']


In [48]:
def proc_sess(exp, period, ev):
    if len(np.unique(ev['subject'])) != 1:
        raise ValueError('Invalid subject ' + str(np.unique(ev['subject'])))
    if len(np.unique(ev['session'])) != 1:
        raise ValueError('Invalid session ' + str(np.unique(ev['session'])))
    subj = ev['subject'][0]
    if subj in bad_subjects:
        return
    sess = ev['session'][0]
    pdir = pow_params[exp][period]['output_path']
    pfile_path = os.path.join(pdir,'pow_'+subj+'_'+str(sess)+'_allleads.hdf5')
    if not os.path.exists(pdir):
        try:
            os.makedirs(pdir)
        except OSError as e:
            print('no problem', e)
            pass
    elif os.path.exists(pfile_path):
        # print pfile_path,'exists!'
        return
    try:
        pfile = h5py.File(pfile_path, 'w-', libver='latest')
    except IOError:
        print('Cannot create', pfile_path)
        return
    print('Processing exp, period, subject, session: ', exp, period, subj, sess)

    hdf5 = True
    if ev['eegfile'][0].split('.')[-1] != 'h5':
        try:
            pairs_path = jr.get_value('pairs', subject=subj,
                                      experiment=exp, session=sess)
        except ValueError as e:
            print(e)
            pfile.close()
            print('removing', pfile_path)
            os.remove(pfile_path)
            return
        tal_reader = TalReader(filename=pairs_path)
        channels = tal_reader.get_monopolar_channels()
        hdf5 = False
    else:
        channels = np.array([])
    if pow_params[exp][period]['mirror']:
        dat = EEGReader(events=ev.view(np.recarray),
                        channels=channels,
                        start_time=pow_params[exp][period]['start_time'],
                        end_time=pow_params[exp][period]['end_time'],
                        buffer_time=0.0).read()
        dat = dat.add_mirror_buffer(pow_params[exp][period]['buf'])
    else:
        dat = EEGReader(events=ev.view(np.recarray),
                        channels=channels,
                        start_time=pow_params[exp][period]['start_time'],
                        end_time=pow_params[exp][period]['end_time'],
                        buffer_time=pow_params[exp][period]['buf']).read()
    if not 'bipolar_pairs' in dat.coords:
        if hdf5:
            # need to figure out how to get bipolar here!
            return
        dat = MonopolarToBipolarMapper(
            time_series=dat, bipolar_pairs=tal_reader.get_bipolar_pairs()).filter()
    if hdf5:
        channels = dat.bipolar_pairs.values
    # set mean to zero to reduce edge effects / ringing in later
    # processing steps:
    dat -= dat.mean('time')
    # Notch Butterworth filter for 60Hz line noise:
    dat = ButterworthFilter(time_series=dat, freq_range=LINEFILT,
                            filt_type='stop', order=4).filter()
    dat, _ = MorletWaveletFilter(
        time_series=dat, freqs=FREQS, output='power', frequency_dim_pos=0,
        width=5, verbose=True).filter()
    dat = np.log10(dat)
    dat = TimeSeries(dat, coords=dat.coords)
    dat = ResampleFilter(time_series=dat, resamplerate=POWHZ).filter()
    dat = dat.remove_buffer(duration=pow_params[exp][period]['buf'])
    data = pfile.create_dataset(
        'data', data=dat.values, compression='gzip', compression_opts=9)
    # finalize HDF5 file:
    for param in pow_params[exp][period]:
        try:
            data.attrs[param] = pow_params[exp][period][param]
        except TypeError:
            data.attrs[param] = [str.encode(s) for s in pow_params[exp][period][param]]
    data.attrs['period'] = period
    try:
        data.attrs['subject'] = subj
    except TypeError:
        data.attrs['subject'] = str.encode(subj)
    data.attrs['session'] = sess
    data.attrs['samplerate'] = np.float(dat.samplerate)
    try:
        data.attrs['exp'] = exp
    except TypeError:
        data.attrs['exp'] = str.encode(exp)
    if not hdf5:
        pfile['tal_struct/tag_name'] = [
            str.encode(tn) for tn in tal_reader.tal_struct_array['tagName']]
        atlases = {}
        for chan in tal_reader.tal_struct_array['atlases']:
            # make sure we get all atlases across channels
            for atls in chan:
                if not atls in atlases:
                    atlases[atls] = {key: [] for key in atlas_defaults}   
        for chan in tal_reader.tal_struct_array['atlases']:
            for atls in atlases:
                for key in atlas_defaults:
                    if atls in chan:
                        if key in chan[atls]:
                            try:
                                val = atlas_defaults[key][1](chan[atls][key])
                            except TypeError:
                                val = atlas_defaults[key][0]
                            atlases[atls][key].append(val)
                        else:
                            atlases[atls][key].append(atlas_defaults[key][0])
                    else:
                        atlases[atls][key].append(atlas_defaults[key][0])
        for atls in atlases:
            for key in atlases[atls]:
                pfile['tal_struct/atlases/'+atls+'/'+key] = atlases[atls][key]
        pfile['tal_struct/code'] = [
            str(c) for c in tal_reader.tal_struct_array['code']]
        pfile['tal_struct/channel_1'] = tal_reader.tal_struct_array['channel_1']
        pfile['tal_struct/channel_2'] = tal_reader.tal_struct_array['channel_2']
        if 'is_explicit' in tal_reader.tal_struct_array.dtype.names:
            pfile['tal_struct/is_explicit'] = tal_reader.tal_struct_array['is_explicit']
        pfile['tal_struct/type_1'] = [
            str(t) for t in tal_reader.tal_struct_array['type_1']]
        pfile['tal_struct/type_2'] = [
            str(t) for t in tal_reader.tal_struct_array['type_2']]
        if 'is_stim_only' in tal_reader.tal_struct_array.dtype.names:
            pfile['tal_struct/is_stim_only'] = tal_reader.tal_struct_array['is_stim_only']
        if 'id' in tal_reader.tal_struct_array.dtype.names:
            pfile['tal_struct/id'] = [
                str(i) for i in tal_reader.tal_struct_array['id']]
        pfile['tal_struct/channel'] = np.array(
            [np.int16(c) for c in tal_reader.tal_struct_array['channel']])
        pfile['tal_struct/eType'] = [
            str(t) for t in tal_reader.tal_struct_array['eType']]
        if hasattr(tal_reader, 'struct_type'):
            pfile['tal_struct/struct_type'] = tal_reader.struct_type
        pfile['tal_struct/struct_name'] = tal_reader.struct_name
        pfile['tal_struct/filename'] = tal_reader.filename
    else:
        h5group = pfile.create_group('h5info')
        h5dat = h5py.File(ev['eegfile'][0], 'r')
        h5dat.copy('bipolar_info', h5group)
        h5dat.copy('names', h5group)
        h5dat.copy('ports', h5group)
        h5dat.copy('sense_channel_info', h5group)
        
    data.dims[0].label = 'freqs'
    pfile['freqs'] = dat.frequency.values
    data.dims.create_scale(pfile['freqs'], 'freqs')
    data.dims[0].attach_scale(pfile['freqs'])

    data.dims[1].label = 'channels'
    pfile['channels_orig'] = channels # possibly monopolar
    if not hdf5:
        pfile['channels'] = pfile['tal_struct/id']
    else:
        pfile['channels'] = pfile['h5info/sense_channel_info/contact_name']
    # bipolar:
    data.dims.create_scale(pfile['channels'], 'channels')
    data.dims[1].attach_scale(pfile['channels'])
    #
    data.dims[3].label = 'time'
    pfile['time'] = dat['time'].values
    data.dims.create_scale(pfile['time'], 'time')
    data.dims[3].attach_scale(pfile['time'])
    data.dims[2].label = 'events'
    events = dat['events'].values
    for dtn in events.dtype.names:
        if events[dtn].dtype == np.object:
            continue
        if events[dtn].dtype.char == 'U':
            pfile['events/'+dtn] = [str.encode(e) for e in events[dtn]]
        else:
            pfile['events/'+dtn] = events[dtn]
        data.dims.create_scale(pfile['events/'+dtn], dtn)
        data.dims[2].attach_scale(pfile['events/'+dtn])
    pfile.close()
    # make file read only to avoid accidental loss:
    os.chmod(pfile_path,0o444)




In [49]:
for exp in ['catFR1']:  # pow_params:
    ev1 = []
    ev2 = []
    ev3 = []
    for f in jr.aggregate_values('task_events', experiment=exp):
        tmpev = BaseEventReader(filename=f).read()
        if 'recognized' in tmpev.dtype.names:
            ev1.append(tmpev)
        elif 'phase' in tmpev.dtype.names:
            ev2.append(tmpev)
        else:
            ev3.append(tmpev)
    ev1 = np.concatenate(ev1)
    ev2 = np.concatenate(ev2)
    ev3 = np.concatenate(ev3)
    for period in ['encoding']: #pow_params[exp]:
        ev = np.concatenate([ev1[list(ev3.dtype.names)],
                             ev2[list(ev3.dtype.names)], ev3])
        # ev = np.concatenate(
        #     [BaseEventReader(filename=f).read() for f
        #      in jr.aggregate_values('task_events',
        #                             experiment=exp)])
        # ev = np.load(pow_params[exp][period]['ev_file'])
        evfilt = np.array([e in pow_params[exp][period]['ev_type']
                           for e in ev['type']])
        ev = ev[evfilt]
        for subj in ['R1374T', 'R1375C']:  # np.unique(ev['subject']):
            subjfilt = ev['subject']==subj
            for sess in np.unique(ev[subjfilt]['session']): #sessions:
                sessfilt = ev['session'] == sess
                if num_mp_procs > 0:
                    res.append(po.apply_async(
                        proc_sess, [exp, period, ev[subjfilt & sessfilt]]))
                else:
                    proc_sess(exp, period, ev[subjfilt & sessfilt])
    


Processing exp, period, subject, session:  catFR1 encoding R1374T 0
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/0/ephys/current_processed/noreref/sources.json
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/0/ephys/current_processed/noreref/../sources.json
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/0/ephys/current_processed/noreref/sources.json
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/0/ephys/current_processed/noreref/../sources.json
total time wavelet loop:  48.152819871902466
230
Processing exp, period, subject, session:  catFR1 encoding R1374T 1
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/1/ephys/current_processed/noreref/sources.json
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/1/ephys/current_processed/noreref/../sources.json
looking for /protocols/r1/subjects/R1374T/experiments/catFR1/sessions/1/ephys/current_processed/noreref/